In [33]:
import pandas as pd
import sqlite3
from datetime import datetime as dt
import hashlib
import sys
import os
from pathlib import Path
file = Path(os.getcwd()).resolve()
parent = file.parent
sys.path.append(str(parent))

from workers import sql_stooq as sql


In [34]:
dat=pd.read_csv("tab3.csv")
tab = "INDEXES"
db_file = '../stooq2.sqlite'
dat['tab']=tab

In [35]:
dat["HASH"] = [
        hashlib.md5("".join(r).encode("utf-8")).hexdigest()
        for r in dat.loc[:, ["symbol", "name", "tab"]].to_records(index=False)
    ]
dat.drop(columns=['tab'], inplace=True)

In [63]:
dat_desc = dat.groupby(['HASH','symbol','name'], group_keys=False).apply(lambda x: x)
dat_desc['from_date']=dat.groupby('HASH')['date'].min().to_list()
dat_desc['end_date']=dat.groupby('HASH')['date'].max().to_list()
dat_desc.drop(columns='date', inplace=True)

In [64]:
dat_desc

,symbol,name,val,HASH,from_date,end_date
0,^AOR,ALL ORDINARIES INDEX - AUSTRALIA,7665.200,54d6670e3cc445ff967dba78949c8387,2023-02-09,2023-02-09
1,^HSI,HANG SENG INDEX - HONG KONG,21624.360,688dbc45deafc4d824c793612c921cda,2023-02-09,2023-02-09
2,^JCI,JAKARTA COMPOSITE INDEX - INDONESIA,6897.370,749e72892680634099072597bf6259fd,2023-02-09,2023-02-09
3,^KLCI,KUALA LUMPUR COMPOSITE INDEX - MALAYSIA,1464.640,b2d4472ebac1583210ff3190b64dee07,2023-02-10,2023-02-10
4,^KOSPI,KOSPI INDEX - SOUTH KOREA,2481.520,361a858db5c45f8388dc4e4ea0392a18,2023-02-09,2023-02-09
5,^NKX,NIKKEI 225 - JAPAN,27584.350,0c2145dd1be5ecaf2ce247384f78e315,2023-02-09,2023-02-09
6,^NZ50,NZX 50 - NEW ZEALAND,12151.480,e0a4491db249e6adb29fe4135d1461b6,2023-02-09,2023-02-09
7,^PSEI,PSEI INDEX - PHILIPPINES,6878.250,3344a7604d48ebbc41c406e1606aeb7b,2023-02-09,2023-02-09
8,^SET,SET INDEX - THAILAND,1669.170,89c4a8b3eeba7b8f8b13da3cf79867e0,2023-02-09,2023-02-09
9,^SHBS,SHANGHAI B-SHARE INDEX - CHINA,296.097,9ededeb148d18ad95a0343510a4456c4,2023-02-09,2023-02-09


In [4]:
sql_columns = sql.tab_columns(tab,db_file)
match_columns = [col in sql_columns for col in dat.columns]
if not all(match_columns):
    print(f"Not known column {list(dat.columns[[not c for c in match_columns]])} in table. Can not write to sql.")

In [5]:
records = list(dat.astype('string').to_records(index=False, column_dtypes=pd.StringDtype))

In [6]:
cmd=[f"""INSERT OR REPLACE INTO {tab} {tuple(dat.columns)}
VALUES {values_list}
""" for values_list in records]

In [13]:
sql.execute_sql(["PRAGMA foreign_keys = ON"], db_file)

{}

In [30]:
sql.execute_sql(["PRAGMA foreign_keys"], db_file)


{'PRAGMA foreign_keys':    foreign_keys
 0             0}

In [29]:
con = sqlite3.connect(
            db_file, 
            detect_types=sqlite3.PARSE_COLNAMES | sqlite3.PARSE_DECLTYPES
        )
cur = con.cursor()
con.execute("PRAGMA foreign_keys = ON")
con.commit()
cur.execute('PRAGMA foreign_keys')
c=cur.fetchall()
con.close()
c

[(1,)]

In [23]:
con = sqlite3.connect(
            db_file, 
            detect_types=sqlite3.PARSE_COLNAMES | sqlite3.PARSE_DECLTYPES
        )
cur = con.cursor()
cur.execute("PRAGMA foreign_keys = ON")
con.commit()
cur.execute('CREATE TABLE INDEXES(HASH TEXT, symbol TEXT, name TEXT, vol INTEGER, val INTEGER, date TIMESTAMP, FOREIGN KEY(HASH) REFERENCES INDEXES_DESC(HASH))')
con.commit()
con.close()

In [26]:
con = sqlite3.connect(
            db_file, 
            detect_types=sqlite3.PARSE_COLNAMES | sqlite3.PARSE_DECLTYPES
        )
cur = con.cursor()
cur.execute(cmd[1])
con.commit()
con.close()


In [41]:
con = sqlite3.connect(
            db_file, 
            detect_types=sqlite3.PARSE_COLNAMES | sqlite3.PARSE_DECLTYPES
        )
cur = con.cursor()
cur.execute(cmd[-1])
cur.execute("SELECT name FROM INDEXES")
cur.fetchall()


[('ALL ORDINARIES INDEX - AUSTRALIA',),
 ('ALL ORDINARIES INDEX - AUSTRALIA',),
 ('HANG SENG INDEX - HONG KONG',),
 ('TAIEX INDEX - TAIWAN',)]

In [43]:
con.commit()